# Inside the black box of A/B testing

# 0. Imports

In [ ]:
import numpy as np
from scipy.stats import bernoulli, norm, binom
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from PIL import Image
import imageio
#import math

# 1. Distribution under the null hypothesis

In [261]:
alpha = 0.05 # significance level
num_replications = 100000 # no. of Monte Carlo replications to be used

The following function calculates the test statistics.

In [3]:
def t_test(num_heads_A: int, n_A: int, num_heads_B: int, n_B: int, alpha: float=.05) -> (float, bool):
    """"On basis of the number of heads in X_1,...,X_{n_A} and Y_1,...,Y_{n_B}
    and the significance level alpa, this function determines the realization of the
    t-test (t) and the outcome (phi) of the test."""
    hat_p_A = num_heads_A / n_A
    hat_p_B = num_heads_B / n_B
    if max(hat_p_A, hat_p_B) < 1 and min(hat_p_A, hat_p_B) > 0:
        t = (hat_p_B - hat_p_A) / np.sqrt(hat_p_A * (1 - hat_p_A) / n_A + hat_p_B * (1 - hat_p_B) / n_B)
    elif ((hat_p_A == 0) and (hat_p_B == 0)) or ((hat_p_A == 1) and (hat_p_B == 1)):
        t = 0
    elif hat_p_B == 1 and hat_p_A == 0: 
        t = np.inf
    elif hat_p_A == 1 and hat_p_B == 0:
        t = - np.inf
    else:
        t = 0
    phi = (t >= norm.ppf(1 - alpha))
    return t, phi

In [245]:
# Example:
t_test(10, 50, 30, 40, alpha)

(6.192884828895332, True)

The following function can be used to simulate datasets and the corresponding realizations of the test statistics. This will be used for the Monte Carlo illustrations.

In [279]:
def simulate_distribution(p_A: float, n_A: int, p_B: float, n_B: int, alpha: float, num_replications: int):
    """This function simulaties \sum_{i=1}^{n_A} X_i and \sum_{i=1}^{n_B} Y_i using i.i.d. draws from a Bernoulli(p_A) 
    and Bernoulli(p_B) distribution respectively and calculates the outcome of the test. This process is repeated 
    num_replications times and alpha is used as significance level."""
    no_heads_A = binom.rvs(n_A, p_A, loc=0, size=num_replications) 
    no_heads_B = binom.rvs(n_B, p_B, loc=0, size=num_replications)
    vectorized_t_test = np.vectorize(t_test)
    t, phi = vectorized_t_test(no_heads_A, n_A, no_heads_B, n_B, alpha)
    return t, phi

In [252]:
# Example:
simulate_distribution_phat(.2, 50, .3, 40, alpha, 10)

(array([-0.32127391,  2.08241782,  2.33398175,  1.83998134,  1.32501674,
        -0.15020762,  0.85857867,  1.06638129, -0.76299831,  2.0773648 ]),
 array([False,  True,  True,  True, False, False, False, False, False,
         True]))

In [265]:
def figure_to_image(fig):
    """Auxiliary function that converts Matplot figure to image"""
    fig.canvas.draw ( )
    w, h = fig.canvas.get_width_height()
    buffer = np.fromstring(fig.canvas.tostring_argb(), dtype=np.uint8)
    buffer.shape = (w, h, 4)
    buffer = np.roll(buffer, 3, axis=2)
    w, h, d = buffer.shape
    return Image.frombuffer("RGBA", (w, h), buffer.tostring())   

In [76]:
#from matplotlib.backends.backend_agg import FigureCanvasAgg
#from matplotlib.figure import Figure

In [277]:
def null_distribution(n_A: int, n_B: int, p_A: float, alpha: float, num_replications: int):
    
    p_B = p_A
    t, phi = simulate_distribution(p_A, n_A, p_B, n_B, alpha, num_replications)
    print(f"The power is {np.mean(phi)}.")
    fig, ax = plt.subplots(1, 1, constrained_layout=True)
    ax.set_title(f"""Distribution of $t$, on basis of {num_replications} Monte Carlo replications and
    for $n_A=n_B$={n_A} and $p_A=p_B=${p_A} versus the N(0, 1) distribution.""")
    ax.set_ylim([0, .6])
    ax.set_xlim([-4, 4])
    x = np.linspace(-4, 4, 100)
    y = stats.norm.pdf(x)
    ax.plot(x, y, "r", lw=2)
    sns.set_theme()
    ax = sns.distplot(t, kde=False, norm_hist=True)
    plt.close()
    return figure_to_image(fig)

In [291]:
imgs_null = []
p_A = .2
sample_sizes = [25, 50, 100, 1000, 5000, 10000]
for n in sample_sizes:
    imgs_null.append(null_distribution(n, n, p_A, alpha, num_replications))
imageio.mimsave("null_distr.gif", imgs_null, format="gif", fps=.5)

KeyboardInterrupt: 

# 2. Distribution under the alternative

In [293]:
def MC_power(n: int, fraction_A: float, p_A: float, p_B: float, alpha: float, num_replications: int):
    """"This functions calculates the power, using Monte Carlo simulations, and presents the distribution
    of the test statistic as well as its asymptotics approximation."""
    n_A = np.int(fraction_A * n)
    n_B = n - n_A
    t, phi = simulate_distribution_phat(p_A, n_A, p_B, n_B, alpha, num_replications)
    fig, ax = plt.subplots(1, 1, constrained_layout=True)
    fig.set_size_inches(11,8)
    ax.set_title(f"""Distribution of $t$, on basis of {num_replications} Monte Carlo replications and for $p_A$={p_A}, $p_B$={p_B} n={n}, and $\lambda$={fraction_A}, 
    its normal approximation (blue) and the (approximating) distribution under the null (in red).""")
    ax.set_ylim([0, .6])
    ax.set_xlim([-4, 8])
    x = np.linspace(-4, 8, 100)
    y = stats.norm.pdf(x)
    ax.plot(x, y, "r", lw=2) # approximating distribution under null
    ax = sns.distplot(t, kde=False, norm_hist=True)
    print(f"""The power of the test is {np.round(np.mean(phi), 2)}. This approximation is based upon
         {num_replications} Monte Carlo replications.""") 
    # check on validity approximations using local asymptotics:
    h = np.sqrt(n) * (p_B - p_A) # parameter for local asymptotics
    mean_local_alternative = h / np.sqrt(p_A * (1 - p_A) / fraction_A + p_B * (1 - p_B) / (1 - fraction_A))
    x = np.linspace( - 4, 8, 100)
    y = stats.norm.pdf(x, loc=mean_local_alternative, scale=1)                                                         
    ax.plot(x, y, "b", lw=2)
    plt.fill_between(x[x > norm.ppf(1 - alpha)], norm.pdf(x[x > norm.ppf(1 - alpha)], loc=mean_local_alternative))
    p = 1 - norm.cdf(norm.ppf(1 - alpha), loc=mean_local_alternative, scale=1)
    print(f"An approximation to power, using local asymptotics, is {np.round(p, 2)}.")
    plt.close()
    return figure_to_image(fig)

In [295]:
imgs_alt = []
p_A = .2
p_B = .23
sample_sizes = [50, 100, 200, 2000, 10000, 20000, 200000]
eta = .5
for n in sample_sizes:
    imgs_alt.append(MC_power(n, eta, p_A, p_B, alpha, num_replications))
imageio.mimsave("alt_distr.gif", imgs_alt, format="gif", fps=.5)


D:\Anaconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


The power of the test is 0.09. This approximation is based upon
         100000 Monte Carlo replications.
An approximation to power, using local asymptotics, is 0.08.


<ipython-input-265-d8f52020b5ef>:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  buffer = np.fromstring(fig.canvas.tostring_argb(), dtype=np.uint8)
<ipython-input-265-d8f52020b5ef>:9: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return Image.frombuffer("RGBA", (w, h), buffer.tostring())
D:\Anaconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


The power of the test is 0.11. This approximation is based upon
         100000 Monte Carlo replications.
An approximation to power, using local asymptotics, is 0.1.


<ipython-input-265-d8f52020b5ef>:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  buffer = np.fromstring(fig.canvas.tostring_argb(), dtype=np.uint8)
<ipython-input-265-d8f52020b5ef>:9: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return Image.frombuffer("RGBA", (w, h), buffer.tostring())
D:\Anaconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


...

<ipython-input-265-d8f52020b5ef>:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  buffer = np.fromstring(fig.canvas.tostring_argb(), dtype=np.uint8)
<ipython-input-265-d8f52020b5ef>:9: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return Image.frombuffer("RGBA", (w, h), buffer.tostring())
D:\Anaconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


The power of the test is 1.0. This approximation is based upon
         100000 Monte Carlo replications.
An approximation to power, using local asymptotics, is 1.0.


<ipython-input-265-d8f52020b5ef>:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  buffer = np.fromstring(fig.canvas.tostring_argb(), dtype=np.uint8)
<ipython-input-265-d8f52020b5ef>:9: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return Image.frombuffer("RGBA", (w, h), buffer.tostring())
D:\Anaconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


The power of the test is 1.0. This approximation is based upon
         100000 Monte Carlo replications.
An approximation to power, using local asymptotics, is 1.0.


<ipython-input-265-d8f52020b5ef>:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  buffer = np.fromstring(fig.canvas.tostring_argb(), dtype=np.uint8)
<ipython-input-265-d8f52020b5ef>:9: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return Image.frombuffer("RGBA", (w, h), buffer.tostring())


In [ ]:
#!brew install imagemagick

# Test statistic





<b>Asymptotic distribution under local alternatives</b> To exploit asymptotic power functions, we introduce the local alternatives $H_a:\, p_B=p_A + h/\sqrt{n}$ and we impose $n_A =[\lambda n]$ and $n_B=[(1-\lambda)n]$, with $\lambda\in (0,1)$. Then we have, under the local alternative $h$ and for large $n$, $t \approx N(\tau,1)$
with $\tau = h / \sqrt{\hat{p}_A(1-\hat{p}_A)/\lambda + \hat{p}_B(1-\hat{p}_B)/(1-\lambda) }$. 


## Stuff for Monte Carlo

## Widget for analyzing power of test

For given settings we calculate the power of the test, i.e. the probability of rejecting the null hypothesis. Two methods are employed: i) Monte Carlo simulations yielding an approximation to the finite-sample distribution of the test statistic, ii) an approximation based on asymptotic theory.

In [ ]:
widgets.interact(power_MC, n=(100, 5000, 50), n_fraction_A=(.25,.75, .05), p_A=(0.01,0.5,0.01), p_B=(0.01,0.5,0.01), alpha=fixed(alpha), no_replications=fixed(no_replications))  

## Calculating required sample size A/B test

In [320]:
def required_total_sample_size(fraction_A: float, p_A: float, p_B: float, desired_power: float, alpha: float=.05) -> int:
    """"This functions determines the sample size n that one needs to achieve desired_power as power. Note that p_B > p_A 
    is required. fraction_A represent the fraction of observations that will come from population A."""
    term_1 =  norm.ppf(1 - alpha) - norm.ppf(1 - desired_power)
    term_2 = np.sqrt(p_A * (1 - p_A) / fraction_A + p_B * (1 - p_B) / (1 - fraction_A)) / (p_B - p_A)
    n = np.int(math.pow(term_1 * term_2, 2))
    return n 

In [324]:
def required_total_sample_size(fraction_A: float, p_A: float, p_B: float, desired_power: float, alpha: float=.05) -> int:
    """"This functions determines the sample size n that one needs to achieve desired_power as power. Note that p_B > p_A 
    is required."""
    term_1 =  norm.ppf(1 - alpha) - norm.ppf(1 - desired_power)
    term_2 = np.sqrt(p_A * (1 - p_A) / fraction_A + p_B * (1 - p_B) / (1 - fraction_A)) / (p_B - p_A)
    n = int(np.ceil(math.pow(term_1 * term_2, 2)))
    print(f"""
    Consider a one-sided level alpha={alpha} test with {100*fraction_A}% of the total no. of observations belonging to
    group A. Then we need {n} observations to detect alternative p_B={p_B} against null p_A=p_B, with 
    p_A={p_A}, with power={desired_power}""")
    return n 

In [325]:
# Example:
fraction_A = .5 
p_A = .2
p_B = .23
power_detecting_min_p_B = .80 # required powe to detect alternative p_B
alpha = .05 # asymptotic size of test
n_req = required_total_sample_size(fraction_A, p_A, p_B, power_detecting_min_p_B, alpha)


    Consider a one-sided level alpha=0.05 test with 50.0% of the total no. of observations belonging to
    group A. Then we need 4632 observations to detect alternative p_B=0.23 against null p_A=p_B, with 
    p_A=0.2, with power=0.8


Check that our calculation is valid. Please note that calculation of the sample size is based on asymptotic approximations, so the desired power needs to be compared with the asymptotic result. It is also important to check if, for the chosen settings, the asymptotic approximations are close to the actual finite-sample results. 

In [327]:
img = MC_power(n_req, fraction_A, p_A, p_B, alpha, num_replications)

D:\Anaconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


The power of the test is 0.8. This approximation is based upon
         100000 Monte Carlo replications.
An approximation to power, using local asymptotics, is 0.8.


<ipython-input-265-d8f52020b5ef>:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  buffer = np.fromstring(fig.canvas.tostring_argb(), dtype=np.uint8)
<ipython-input-265-d8f52020b5ef>:9: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return Image.frombuffer("RGBA", (w, h), buffer.tostring())


In [328]:
img.save("sample_size_verified.gif")